# Install Libraries

In [3]:
#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup


# Test Geolocator

In [13]:


#address = 'Charlotte, NC'
address = 'One Paul Brown Stadium, Cincinnati, OH 45202'
geolocator = Nominatim(user_agent="stad_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates are {}, {},{}.'.format(latitude, longitude, location))



AttributeError: 'NoneType' object has no attribute 'latitude'

# Get Stadium List

In [50]:

url='https://raw.githubusercontent.com/ptdanner/Coursera_Capstone/master/NFLStadiums2.csv'
nfl_df=pd.read_csv(url,error_bad_lines=False)
nfl_df

,Team,Division,Stadium,Capacity,Address
0,Buffalo Bills,AFC East,Bills Stadium,"71,608",One Bills Drive: Orchard Park: NY 14127
1,Miami Dolphins,AFC East,Hard Rock Stadium,"65,326",2201 NW 196th Terrace: Miami Gardens: FL 33056
2,New England Patriots,AFC East,Gillette Stadium,"66,829",1 Patriot Pl: Foxborough: MA 02035
3,New York Jets,AFC East,MetLife Stadium,"82,500",1 MetLife Stadium Dr: East Rutherford: NJ 07073
4,Baltimore Ravens,AFC North,M&T Bank Stadium,"71,008",1101 Russell Street: Baltimore: MD 21230
5,Cincinnati Bengals,AFC North,Paul Brown Stadium,"65,515",Cincinnati: OH 45202
6,Cleveland Browns,AFC North,FirstEnergy Stadium,"67,895",Cleveland: OH 44114
7,Pittsburgh Steelers,AFC North,Heinz Field,"68,400",Pittsburgh: PA 15212
8,Houston Texans,AFC South,NRG Stadium,"72,220",NRG Pkwy: Houston: TX 77054
9,Indianapolis Colts,AFC South,Lucas Oil Stadium,"67,000",500 S Capitol Ave: Indianapolis: IN 46225


# Ckean Stadium List Address Column

In [51]:
nfl_df['Address'] = nfl_df['Address'].str.replace(':', ',')
nfl_df.head(10)

,Team,Division,Stadium,Capacity,Address
0,Buffalo Bills,AFC East,Bills Stadium,"71,608","One Bills Drive, Orchard Park, NY 14127"
1,Miami Dolphins,AFC East,Hard Rock Stadium,"65,326","2201 NW 196th Terrace, Miami Gardens, FL 33056"
2,New England Patriots,AFC East,Gillette Stadium,"66,829","1 Patriot Pl, Foxborough, MA 02035"
3,New York Jets,AFC East,MetLife Stadium,"82,500","1 MetLife Stadium Dr, East Rutherford, NJ 07073"
4,Baltimore Ravens,AFC North,M&T Bank Stadium,"71,008","1101 Russell Street, Baltimore, MD 21230"
5,Cincinnati Bengals,AFC North,Paul Brown Stadium,"65,515","Cincinnati, OH 45202"
6,Cleveland Browns,AFC North,FirstEnergy Stadium,"67,895","Cleveland, OH 44114"
7,Pittsburgh Steelers,AFC North,Heinz Field,"68,400","Pittsburgh, PA 15212"
8,Houston Texans,AFC South,NRG Stadium,"72,220","NRG Pkwy, Houston, TX 77054"
9,Indianapolis Colts,AFC South,Lucas Oil Stadium,"67,000","500 S Capitol Ave, Indianapolis, IN 46225"


# Get Shape of Stadium List

In [52]:
count_row = nfl_df.shape[0]
print(count_row)

32


# Get Latitude and Longitude for Stadiums

In [53]:
# Find longitude and latitude values for every Stadium Postal address. We will be brave enough to use the Geocoder package...

geolocator = Nominatim(user_agent="stad_explorer")

for i in range(len(nfl_df)):
    address = nfl_df.loc[i,'Address']
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        nfl_df.loc[i,'latitude']=location.latitude
        nfl_df.loc[i,'longitude']=location.longitude
        print('The geograpical coordinates are {}, {},{}.'.format(location, latitude, longitude))

    except:
        print("Exception on: {} ".format(address)) #Look for Addresses with missing coordinates
        pass
        


The geograpical coordinates are Bills Drive, Orchard Park, Erie County, New York, 14127, United States of America, 42.7713413,-78.7873628.
The geograpical coordinates are 2201, Northwest 196th Terrace, Miami Gardens, Miami-Dade County, Florida, 33056, United States of America, 25.95379015,-80.23816387657178.
The geograpical coordinates are 1, Patriot Place, Patriot Place North Marketplace, Foxborough, Norfolk County, Massachusetts, 02035, United States of America, 42.0922722,-71.26546383597562.
The geograpical coordinates are MetLife Stadium, Road D, East Rutherford, Bergen County, New Jersey, 07072, United States of America, 40.81350705,-74.07434417766939.
The geograpical coordinates are M&T Bank Stadium, 1101, Russell Street, Ridgely's Delight, Baltimore, Maryland, 21230, United States of America, 39.2779664,-76.62380930835468.
The geograpical coordinates are Dayton Cincinnati Pike, Montgomery County, Ohio, 45342, United States of America, 39.594652,-84.283978.
The geograpical coordi

In [9]:
nfl_df

,Team,Division,Stadium,Capacity,Address,latitude,longitude
0,Buffalo Bills,AFC East,Bills Stadium,"71,608","One Bills Drive, Orchard Park, NY 14127",42.771341,-78.787363
1,Miami Dolphins,AFC East,Hard Rock Stadium,"65,326","2201 NW 196th Terrace, Miami Gardens, FL 33056",25.953790,-80.238164
2,New England Patriots,AFC East,Gillette Stadium,"66,829","1 Patriot Pl, Foxborough, MA 02035",42.092272,-71.265464
3,New York Jets,AFC East,MetLife Stadium,"82,500","1 MetLife Stadium Dr, East Rutherford, NJ 07073",40.813507,-74.074344
4,Baltimore Ravens,AFC North,M&T Bank Stadium,"71,008","1101 Russell Street, Baltimore, MD 21230",39.277966,-76.623809
5,Cincinnati Bengals,AFC North,Paul Brown Stadium,"65,515","One Paul Brown Stadium, Cincinnati, OH 45202",NaN,NaN
6,Cleveland Browns,AFC North,FirstEnergy Stadium,"67,895","100 Alfred Lerner Way, Cleveland, OH 44114",NaN,NaN
7,Pittsburgh Steelers,AFC North,Heinz Field,"68,400","100 Art Rooney Ave, Pittsburgh, PA 15212",NaN,NaN
8,Houston Texans,AFC South,NRG Stadium,"72,220","NRG Pkwy, Houston, TX 77054",29.686424,-95.407756
9,Indianapolis Colts,AFC South,Lucas Oil Stadium,"67,000","500 S Capitol Ave, Indianapolis, IN 46225",39.762918,-86.161925


In [49]:
#TEST FOR EXCEPTION ADDRESSES DELETE ME LATER
#address = 'Charlotte, NC'
#address = nfl_df['Address'].iloc[1]
address ='302 W 3rd St, Cincinnati, OH 45202'
geolocator = Nominatim(user_agent="stad_explorer")
location = geolocator.geocode(address)
#latitude = location.latitude
#longitude = location.longitude
#print('The geograpical coordinates are {}, {},{}.'.format(latitude, longitude, location))
print('The geograpical coordinates are {}.'.format(location))

The geograpical coordinates are None.


In [54]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# import folium for map rendering library
!pip install folium
import folium 

print('Libraries imported.')

Libraries imported.


# Show Map of all locations

In [55]:
# create map of USA using latitude and longitude values
map_nfl = folium.Map(location=[latitude, longitude], zoom_start=4)

# add markers to map
for lat, lng, team, stadium in zip(nfl_df['latitude'], nfl_df['longitude'], nfl_df['Team'], nfl_df['Stadium']):
    try:
        label = '{}, {}'.format(team, stadium)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_nfl)  
    except:
        pass
map_nfl

# Define Foursquare settings

In [56]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'W15FICLBVNUELUJBYTZBDDIQ0PKD0MTMMTLIE3G0GXT4VSQS' # your Foursquare ID
CLIENT_SECRET = 'YALL5RDJB43015QBZBQ3XYQSN2T1NENRC3PMOYVVRLFZVFKK' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W15FICLBVNUELUJBYTZBDDIQ0PKD0MTMMTLIE3G0GXT4VSQS
CLIENT_SECRET:YALL5RDJB43015QBZBQ3XYQSN2T1NENRC3PMOYVVRLFZVFKK


# Retrieve all Foursquare Categories

In [89]:
#Create URL to retrieve all categories
url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION)
#Send the GET request and examine the results
results_cat = requests.get(url).json()
results_cat

{'meta': {'code': 200, 'requestId': '5fced287b75f0438f1a77385'},
 'response': {'categories': [{'id': '4d4b7104d754a06370d81259',
    'name': 'Arts & Entertainment',
    'pluralName': 'Arts & Entertainment',
    'shortName': 'Arts & Entertainment',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_',
     'suffix': '.png'},
    'categories': [{'id': '56aa371be4b08b9a8d5734db',
      'name': 'Amphitheater',
      'pluralName': 'Amphitheaters',
      'shortName': 'Amphitheater',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_',
       'suffix': '.png'},
      'categories': []},
     {'id': '4fceea171983d5d06c3e9823',
      'name': 'Aquarium',
      'pluralName': 'Aquariums',
      'shortName': 'Aquarium',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/aquarium_',
       'suffix': '.png'},
      'categories': []},
     {'id': '4bf58dd8d48988d1e1931735',
      'name': 'A

In [123]:
# assign relevant part of JSON to venues
categories = results_cat['response']['categories']
categories
# tranform categories into a dataframe
dataframe_cat = pd.json_normalize(categories)
dataframe_cat.head()
# filter for only Food
dataframe_cat_food = dataframe_cat[dataframe_cat['name']=='Food']
dataframe_cat_food
categoriestext = dataframe_cat_food.loc[3,'categories']
categoriestext
dataframe_subcat = pd.json_normalize(categoriestext)
dataframe_subcat.shape
categorylist = dataframe_subcat['name'].tolist()

type(mylist)
categorylist

['Afghan Restaurant',
 'African Restaurant',
 'American Restaurant',
 'Armenian Restaurant',
 'Asian Restaurant',
 'Australian Restaurant',
 'Austrian Restaurant',
 'BBQ Joint',
 'Bagel Shop',
 'Bakery',
 'Bangladeshi Restaurant',
 'Belgian Restaurant',
 'Bistro',
 'Breakfast Spot',
 'Bubble Tea Shop',
 'Buffet',
 'Burger Joint',
 'Cafeteria',
 'Café',
 'Cajun / Creole Restaurant',
 'Caribbean Restaurant',
 'Caucasian Restaurant',
 'Coffee Shop',
 'Comfort Food Restaurant',
 'Creperie',
 'Czech Restaurant',
 'Deli / Bodega',
 'Dessert Shop',
 'Diner',
 'Donut Shop',
 'Dumpling Restaurant',
 'Dutch Restaurant',
 'Eastern European Restaurant',
 'English Restaurant',
 'Falafel Restaurant',
 'Fast Food Restaurant',
 'Fish & Chips Shop',
 'Fondue Restaurant',
 'Food Court',
 'Food Stand',
 'Food Truck',
 'French Restaurant',
 'Fried Chicken Joint',
 'Friterie',
 'Gastropub',
 'German Restaurant',
 'Gluten-free Restaurant',
 'Greek Restaurant',
 'Halal Restaurant',
 'Hawaiian Restaurant',
 '

In [58]:
#DELETEME# function that extracts the categories name and parents
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [59]:
#DELETEME
categories = results_cat['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

KeyError: 'groups'

In [71]:
nfl_df.loc[1,'Team']
stadium_latitude = nfl_df.loc[1, 'latitude'] # stadium latitude value
stadium_longitude = nfl_df.loc[1, 'longitude'] # stadium longitude value

stadium_name = nfl_df.loc[1, 'Stadium'] # stadium name

print('Latitude and longitude values of {} are {}, {}.'.format(stadium_name, 
                                                               stadium_latitude, 
                                                               stadium_longitude))

Latitude and longitude values of Hard Rock Stadium are 25.95379015, -80.23816387657178.


In [87]:
#HERE#####TEST FOR CATEGORY SEARCH
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)
search_query = 'Food'
radius = 1000
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
results
# assign relevant part of JSON to venues
venues = results['response']['venues']
venues
# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

40.7149555 -74.0153365
Food .... OK!


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,49bc3b0af964a52020541fe3,Whole Foods Market,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",v-1607389767,False,270 Greenwich Street,at Warren St,40.715579,-74.011368,"[{'label': 'display', 'lat': 40.71557915542060...",...,NY,United States,"[270 Greenwich Street (at Warren St), New York...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4bbb7d5aafe1b713ac3f2f4b,Food Bank For New York City,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1607389767,False,39 Broadway Fl 10,NaN,40.706336,-74.012972,"[{'label': 'display', 'lat': 40.70633552386926...",...,NY,United States,"[39 Broadway Fl 10, New York, NY 10006, United...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4ded108b52b100c01e04d28b,Biryani House Food Truck,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1607389767,False,140 Broadway,Beaver St Btw Broadway & New St,40.704956,-74.012537,"[{'label': 'display', 'lat': 40.70495624253704...",...,New York,United States,[140 Broadway (Beaver St Btw Broadway & New St...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,51378818e4b079558b0a6a61,Food Cart,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1607389767,False,Greenwich Street,Barclay Street,40.714182,-74.012413,"[{'label': 'display', 'lat': 40.714182, 'lng':...",...,NY,United States,"[Greenwich Street (Barclay Street), New York, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,56904b7e498eaf25c7535429,Food & Wine Magazine,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1607389767,False,225 Liberty St,NaN,40.711875,-74.015287,"[{'label': 'display', 'lat': 40.711875, 'lng':...",...,NY,United States,"[225 Liberty St, New York, NY, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
####CONTINUE TEST
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
dataframe_filtered

,name,categories,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,id
0,Whole Foods Market,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",270 Greenwich Street,at Warren St,40.715579,-74.011368,"[{'label': 'display', 'lat': 40.71557915542060...",341,10007,US,Tribeca,New York,NY,United States,"[270 Greenwich Street (at Warren St), New York...",49bc3b0af964a52020541fe3
1,Food Bank For New York City,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",39 Broadway Fl 10,NaN,40.706336,-74.012972,"[{'label': 'display', 'lat': 40.70633552386926...",980,10006,US,NaN,New York,NY,United States,"[39 Broadway Fl 10, New York, NY 10006, United...",4bbb7d5aafe1b713ac3f2f4b
2,Biryani House Food Truck,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",140 Broadway,Beaver St Btw Broadway & New St,40.704956,-74.012537,"[{'label': 'display', 'lat': 40.70495624253704...",1137,NaN,US,Financial District,NaN,New York,United States,[140 Broadway (Beaver St Btw Broadway & New St...,4ded108b52b100c01e04d28b
3,Food Cart,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",Greenwich Street,Barclay Street,40.714182,-74.012413,"[{'label': 'display', 'lat': 40.714182, 'lng':...",261,NaN,US,NaN,New York,NY,United States,"[Greenwich Street (Barclay Street), New York, ...",51378818e4b079558b0a6a61
4,Food & Wine Magazine,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",225 Liberty St,NaN,40.711875,-74.015287,"[{'label': 'display', 'lat': 40.711875, 'lng':...",342,NaN,US,NaN,New York,NY,United States,"[225 Liberty St, New York, NY, United States]",56904b7e498eaf25c7535429
5,Food & Wine Test Kitchen,"[{'id': '4bf58dd8d48988d171941735', 'name': 'E...",225 Liberty St,NaN,40.711888,-74.015212,"[{'label': 'display', 'lat': 40.711888, 'lng':...",341,NaN,US,NaN,New York,NY,United States,"[225 Liberty St, New York, NY, United States]",56904b22498e43f06a074ac2
6,Food Cart,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",Barclay St,Greenwich St,40.712583,-74.011808,"[{'label': 'display', 'lat': 40.71258340005171...",397,NaN,US,NaN,New York,NY,United States,"[Barclay St (Greenwich St), New York, NY, Unit...",557aa21c498e430a4475bae6
7,Chambers St Halal Food,"[{'id': '56aa371be4b08b9a8d57350b', 'name': 'F...",NaN,NaN,40.716241,-74.010687,"[{'label': 'display', 'lat': 40.716241, 'lng':...",417,10007,US,NaN,New York,NY,United States,"[New York, NY 10007, United States]",5af9ba33ba57b4002cc1baa2
8,Zak's Halal Food,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",Liberty Plaza,NaN,40.712357,-74.014264,"[{'label': 'display', 'lat': 40.71235673019370...",303,NaN,US,NaN,New York,NY,United States,"[Liberty Plaza, New York, NY, United States]",52b1e86e498ed4267316789b
9,South Food Truck Court,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",NaN,Liberty,40.711932,-74.015537,"[{'label': 'display', 'lat': 40.711932, 'lng':...",336,NaN,US,NaN,New York,NY,United States,"[Liberty, New York, NY, United States]",51e57352498eb9115b270d65


In [79]:
####CONTINUE TEST

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
#dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,id
0,Whole Foods Market,Grocery Store,270 Greenwich Street,at Warren St,40.715579,-74.011368,"[{'label': 'display', 'lat': 40.71557915542060...",341,10007,US,Tribeca,New York,NY,United States,"[270 Greenwich Street (at Warren St), New York...",49bc3b0af964a52020541fe3
1,Food Bank For New York City,Office,39 Broadway Fl 10,NaN,40.706336,-74.012972,"[{'label': 'display', 'lat': 40.70633552386926...",980,10006,US,NaN,New York,NY,United States,"[39 Broadway Fl 10, New York, NY 10006, United...",4bbb7d5aafe1b713ac3f2f4b
2,Biryani House Food Truck,Food Truck,140 Broadway,Beaver St Btw Broadway & New St,40.704956,-74.012537,"[{'label': 'display', 'lat': 40.70495624253704...",1137,NaN,US,Financial District,NaN,New York,United States,[140 Broadway (Beaver St Btw Broadway & New St...,4ded108b52b100c01e04d28b
3,Food Cart,Food Truck,Greenwich Street,Barclay Street,40.714182,-74.012413,"[{'label': 'display', 'lat': 40.714182, 'lng':...",261,NaN,US,NaN,New York,NY,United States,"[Greenwich Street (Barclay Street), New York, ...",51378818e4b079558b0a6a61
4,Food & Wine Magazine,Office,225 Liberty St,NaN,40.711875,-74.015287,"[{'label': 'display', 'lat': 40.711875, 'lng':...",342,NaN,US,NaN,New York,NY,United States,"[225 Liberty St, New York, NY, United States]",56904b7e498eaf25c7535429
5,Food & Wine Test Kitchen,Event Space,225 Liberty St,NaN,40.711888,-74.015212,"[{'label': 'display', 'lat': 40.711888, 'lng':...",341,NaN,US,NaN,New York,NY,United States,"[225 Liberty St, New York, NY, United States]",56904b22498e43f06a074ac2
6,Food Cart,Food Truck,Barclay St,Greenwich St,40.712583,-74.011808,"[{'label': 'display', 'lat': 40.71258340005171...",397,NaN,US,NaN,New York,NY,United States,"[Barclay St (Greenwich St), New York, NY, Unit...",557aa21c498e430a4475bae6
7,Chambers St Halal Food,Food Stand,NaN,NaN,40.716241,-74.010687,"[{'label': 'display', 'lat': 40.716241, 'lng':...",417,10007,US,NaN,New York,NY,United States,"[New York, NY 10007, United States]",5af9ba33ba57b4002cc1baa2
8,Zak's Halal Food,Food Truck,Liberty Plaza,NaN,40.712357,-74.014264,"[{'label': 'display', 'lat': 40.71235673019370...",303,NaN,US,NaN,New York,NY,United States,"[Liberty Plaza, New York, NY, United States]",52b1e86e498ed4267316789b
9,South Food Truck Court,Food Truck,NaN,Liberty,40.711932,-74.015537,"[{'label': 'display', 'lat': 40.711932, 'lng':...",336,NaN,US,NaN,New York,NY,United States,"[Liberty, New York, NY, United States]",51e57352498eb9115b270d65


# Retrieve Venues surrounding One Stadium

In [127]:
nfl_df.loc[1,'Team']
stadium_latitude = nfl_df.loc[1, 'latitude'] # stadium latitude value
stadium_longitude = nfl_df.loc[1, 'longitude'] # stadium longitude value

stadium_name = nfl_df.loc[1, 'Stadium'] # stadium name

print('Latitude and longitude values of {} are {}, {}.'.format(stadium_name, 
                                                               stadium_latitude, 
                                                               stadium_longitude))

Latitude and longitude values of Hard Rock Stadium are 25.95379015, -80.23816387657178.


In [128]:
#Get the top 100 venues within a radius of 1000 meters.
#First, let's create the GET request URL. Name your URL url.

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    stadium_latitude, 
    stadium_longitude, 
    radius, 
    LIMIT)
url # display URL

#Send the GET request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fced935e0651225f9de7311'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Miami Gardens',
  'headerFullLocation': 'Miami Gardens',
  'headerLocationGranularity': 'city',
  'totalResults': 38,
  'suggestedBounds': {'ne': {'lat': 25.967290163500014,
    'lng': -80.2231776698248},
   'sw': {'lat': 25.940290136499986, 'lng': -80.25315008331874}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b15500cf964a520c2af23e3',
       'name': 'Hard Rock Stadium',
       'location': {'address': '347 Don Shula Dr',
        'lat': 25.95807361268035,
        'lng': -80.23887276649475,
        'labeledLatLngs': [{'label'

# Put Venues in Dataframe

In [129]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [131]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues


,name,categories,lat,lng
0,Hard Rock Stadium,Football Stadium,25.958074,-80.238873
1,Sonic Beach Miami Gardens,Fast Food Restaurant,25.954844,-80.244629
2,Pollo Tropical,Latin American Restaurant,25.953214,-80.246266
3,LIV Sun Life,Nightclub,25.957879,-80.239208
4,Team Store,Souvenir Shop,25.957387,-80.239077
5,The Mint Lounge,Strip Club,25.944400,-80.243692
6,24 Hour Fitness,Gym / Fitness Center,25.951057,-80.243996
7,Betty T. Ferguson Recreational Complex,Gym,25.954751,-80.251803
8,GameStop,Video Game Store,25.952784,-80.246758
9,Starbucks,Coffee Shop,25.951235,-80.245499


In [138]:
nearby_food = nearby_venues[nearby_venues['categories'].isin(categorylist)]
#rpt[rpt['STK_ID'].isin(stk_list)]
#nearby_food.reset_index(drop = true)
nearby_food

,name,categories,lat,lng
1,Sonic Beach Miami Gardens,Fast Food Restaurant,25.954844,-80.244629
2,Pollo Tropical,Latin American Restaurant,25.953214,-80.246266
9,Starbucks,Coffee Shop,25.951235,-80.245499
13,Subway,Sandwich Place,25.954251,-80.246415
15,Denny's,Breakfast Spot,25.953595,-80.246411
16,Dunkin',Donut Shop,25.954899,-80.246675
18,Quickies,Sandwich Place,25.941941,-80.244859
21,Centerplate,American Restaurant,25.957268,-80.238909
27,Golden Krust Caribbean Restaurant,Caribbean Restaurant,25.953067,-80.246565
31,South Beach Concession,Food Truck,25.952216,-80.227843


In [148]:
nearby_venues.shape

(38, 4)

# Get Venues for All Stadiums

In [149]:

def getNearbyVenues(names,divisions, stadiums, capacities, latitudes, longitudes, radius=1500, limit=100):
    
    venues_list=[]
    for name, division, stadium, capacity, lat, lng in zip(names, divisions, stadiums, capacities, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)

       # url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
       #     CLIENT_ID, 
       #     CLIENT_SECRET, 
       #     VERSION, 
       #     lat, 
       #     lng, 
       #     radius, 
       #     LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            division,
            stadium,
            capacity,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Team',
                  'Divison',
                  'Stadium',
                  'Capacity',
                  'Stadium Latitude', 
                  'Stadium Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [151]:
#Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.
#Get the top 100 venues within a radius of 1000 meters.
#First, let's create the GET request URL. Name your URL url.

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1500 # define radius
stadium_venues = getNearbyVenues(names=nfl_df['Team'],
                                   divisions=nfl_df['Division'],
                                   stadiums=nfl_df['Stadium'],
                                   capacities=nfl_df['Capacity'], 
                                   latitudes=nfl_df['latitude'],
                                   longitudes=nfl_df['longitude'],
                                   radius=1500,
                                   limit = 100
                                  )

Buffalo Bills
Miami Dolphins
New England Patriots
New York Jets
Baltimore Ravens
Cincinnati Bengals
Cleveland Browns
Pittsburgh Steelers
Houston Texans
Indianapolis Colts
Jacksonville Jaguars
Tennessee Titans
Denver Broncos
Kansas City Chiefs
Las Vegas Raiders
Los Angeles Chargers
Dallas Cowboys
New York Giants
Philadelphia Eagles
Washington Redskins
Chicago Bears
Detroit Lions
Green Bay Packers
Minnesota Vikings
Atlanta Falcons
Carolina Panthers
New Orleans Saints
Tampa Bay Buccaneers
Arizona Cardinals
Los Angeles Rams
San Francisco 49ers
Seattle Seahawks


In [152]:
print(stadium_venues.shape)
stadium_venues.head()

(2317, 10)


,Team,Divison,Stadium,Capacity,Stadium Latitude,Stadium Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Buffalo Bills,AFC East,Bills Stadium,"71,608",42.771341,-78.787363,New Era Field (Bills Stadium),42.773684,-78.787230,Football Stadium
1,Buffalo Bills,AFC East,Bills Stadium,"71,608",42.771341,-78.787363,ADPRO Sports Training Center,42.770794,-78.786618,Athletics & Sports
2,Buffalo Bills,AFC East,Bills Stadium,"71,608",42.771341,-78.787363,Bills Tailgating,42.776580,-78.788795,Other Great Outdoors
3,Buffalo Bills,AFC East,Bills Stadium,"71,608",42.771341,-78.787363,The Bills Store,42.774717,-78.788917,Souvenir Shop
4,Buffalo Bills,AFC East,Bills Stadium,"71,608",42.771341,-78.787363,Tailgaters Bar & Grill,42.777837,-78.788939,Bar


In [153]:
nearby_food = stadium_venues[stadium_venues['Venue Category'].isin(categorylist)]
#rpt[rpt['STK_ID'].isin(stk_list)]

nearby_food.reset_index(drop=True, inplace=True)
nearby_food

,Team,Divison,Stadium,Capacity,Stadium Latitude,Stadium Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Buffalo Bills,AFC East,Bills Stadium,"71,608",42.771341,-78.787363,Louie's Texas Red Hots,42.777122,-78.791913,Hot Dog Joint
1,Buffalo Bills,AFC East,Bills Stadium,"71,608",42.771341,-78.787363,La Galleria Restaurant,42.775483,-78.793443,Restaurant
2,Buffalo Bills,AFC East,Bills Stadium,"71,608",42.771341,-78.787363,Tim Hortons,42.777868,-78.791172,Coffee Shop
3,Buffalo Bills,AFC East,Bills Stadium,"71,608",42.771341,-78.787363,SUBWAY,42.778374,-78.790024,Sandwich Place
4,Buffalo Bills,AFC East,Bills Stadium,"71,608",42.771341,-78.787363,Danny's South,42.766996,-78.791909,American Restaurant
...,...,...,...,...,...,...,...,...,...,...
801,Seattle Seahawks,NFC West,Lumen Field,"69,000",47.595346,-122.331644,Hit it Here Cafe,47.591173,-122.331262,American Restaurant
802,Seattle Seahawks,NFC West,Lumen Field,"69,000",47.595346,-122.331644,Starbucks,47.584239,-122.333950,Coffee Shop
803,Seattle Seahawks,NFC West,Lumen Field,"69,000",47.595346,-122.331644,Starbucks,47.607012,-122.335716,Coffee Shop
804,Seattle Seahawks,NFC West,Lumen Field,"69,000",47.595346,-122.331644,Starbucks,47.597098,-122.312493,Coffee Shop
